# 수신된 데이터를 분석함

In [1]:
import asyncio

import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

import nest_asyncio
nest_asyncio.apply()

from Workspace.Receiver.Futures.Public.KlineCycleFetcher import KlineCycleFetcher
from SystemConfig import Streaming
import Workspace.Utils.TradingUtils as tr_utils
import Workspace.Receiver.Futures.Receiver_den_analy as receiver
import Workspace.Analysis.Indicator as indicator

from Workspace.DataStorage.CalculationReport import CalculationReport

report = CalculationReport()

symbol = Streaming.symbols[0]
intervals = Streaming.intervals
queue_async = asyncio.Queue()
instance_kline_fethcer = KlineCycleFetcher(queue_async)
storage = receiver.storage_history

async def fetcher_data():
    for interval in intervals:
        await instance_kline_fethcer.fetch_and_queue(symbol, interval)
        data = await instance_kline_fethcer.queue.get()
        s, i, data = tr_utils.Extractor.unpack_message(data)
        conver_to_interval = f"interval_{i}"
        storage.set_data(s, conver_to_interval, data)
        
asyncio.run(fetcher_data())

### stroage 데이터를 호출하기 위하여 convert_to_interval 값이 필요하다,

In [8]:
import numpy as np
from typing import List
import json

class SMA:
    def __init__(self, stroage, periods:List):
        self.intervals = Streaming.intervals
        self.symbol = Streaming.symbols
        self.convert_to_interval = [f"interval_{i}" for i in self.intervals]
        self.stroage = stroage
        self.periods = periods

        for attr in self.convert_to_interval:
            setattr(self, attr, [])

    def cals(self):
        for i in self.convert_to_interval:
            data = self.stroage.get_data(self.symbol[0], i)
            convert_to_array = np.array(data, float)
            for period in self.periods:
                attr_name = f"ma_sma_btcusdt_{i}_{period}"
                print(attr_name)
                cals = indicator.MA.sma(convert_to_array[:, 4], period)
                report.set_data(attr_name, cals)
            
instance_sma = SMA(storage, [7,25, 99])
instance_sma.cals()

ma_sma_btcusdt_interval_5m_7
ma_sma_btcusdt_interval_5m_25
ma_sma_btcusdt_interval_5m_99
ma_sma_btcusdt_interval_30m_7
ma_sma_btcusdt_interval_30m_25
ma_sma_btcusdt_interval_30m_99
ma_sma_btcusdt_interval_1h_7
ma_sma_btcusdt_interval_1h_25
ma_sma_btcusdt_interval_1h_99
ma_sma_btcusdt_interval_1d_7
ma_sma_btcusdt_interval_1d_25
ma_sma_btcusdt_interval_1d_99


In [9]:
report.get_attr_fields()

['ma_sma_btcusdt_interval_5m_7',
 'ma_sma_btcusdt_interval_5m_25',
 'ma_sma_btcusdt_interval_5m_99',
 'ma_sma_btcusdt_interval_30m_7',
 'ma_sma_btcusdt_interval_30m_25',
 'ma_sma_btcusdt_interval_30m_99',
 'ma_sma_btcusdt_interval_1h_7',
 'ma_sma_btcusdt_interval_1h_25',
 'ma_sma_btcusdt_interval_1h_99',
 'ma_sma_btcusdt_interval_1d_7',
 'ma_sma_btcusdt_interval_1d_25',
 'ma_sma_btcusdt_interval_1d_99']